# Setup/Test Environment:

In [1]:
from web_footprinting_utils import setup_ipython
import os

get_ipython, INTERACTIVE = setup_ipython()

Matplotlib is building the font cache; this may take a moment.


In [2]:
import sys
sys.executable

'/root/miniconda/bin/python'

In [3]:
%env TARGET=10.10.144.120

env: TARGET=10.10.144.120


In [4]:
!echo Make sure we are connected to VPN:
result = !ip a | grep tun0

if not len(result)>0:
    print("Not connected!")
#     input()
else:
    print("Connected!")
    print(result)

Make sure we are connected to VPN:
Not connected!


# Initial (Unreliable) RustScan:

In [5]:
import os
pwd = os.getcwd()
rustscan = "docker run -v '%s':/workspace  --volume /etc/passwd:/etc/passwd:ro --volume /etc/group:/etc/group:ro --user $(id -u) --rm my-other-github-account/rustscan" % pwd

In [6]:
!{rustscan} -t 4000 -r 1-65535 -a {os.environ["TARGET"]} -- -Pn -A -sV

.----. .-. .-. .----..---.  .----. .---.   .--.  .-. .-.
| {}  }| { } |{ {__ {_   _}{ {__  /  ___} / {} \ |  `| |
| .-. \| {_} |.-._} } | |  .-._} }\     }/  /\  \| |\  |
`-' `-'`-----'`----'  `-'  `----'  `---' `-'  `-'`-' `-'
The Modern Day Port Scanner.
________________________________________
: https://discord.gg/GFrQsGy           :
: https://github.com/RustScan/RustScan :
 --------------------------------------
Please contribute more quotes to our GitHub https://github.com/rustscan/rustscan

[~] The config file is expected to be at "/root/.rustscan.toml"
[~] File limit higher than batch size. Can increase speed by increasing batch size '-b 1048476'.
Open 10.10.144.120:22
Open 10.10.144.120:80
[~] Starting Script(s)
[>] Running script "nmap -vvv -p {{port}} {{ip}} -Pn -A -sV" on ip 10.10.144.120
Depending on the complexity of the script, results may take some time to appear.
[~] Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-13 23:52 UTC
NSE: Loaded 151 scripts for scanning.
NS

# Initial Nmap:

In [7]:
!echo $TARGET
!mkdir -p scans/quick scans/tcp scans/udp scans/scripts scans/http scans/no_ping

10.10.144.120


In [8]:
!ping -c 5 $TARGET

PING 10.10.144.120 (10.10.144.120) 56(84) bytes of data.
64 bytes from 10.10.144.120: icmp_seq=1 ttl=64 time=0.461 ms
64 bytes from 10.10.144.120: icmp_seq=2 ttl=64 time=0.288 ms
64 bytes from 10.10.144.120: icmp_seq=3 ttl=64 time=0.332 ms
64 bytes from 10.10.144.120: icmp_seq=4 ttl=64 time=1.09 ms
64 bytes from 10.10.144.120: icmp_seq=5 ttl=64 time=0.398 ms

--- 10.10.144.120 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4081ms
rtt min/avg/max/mdev = 0.288/0.512/1.085/0.292 ms


# Quick Scan:

In [9]:
!nmap -vv -oA scans/quick/quick_scan $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-13 23:53 UTC
Initiating ARP Ping Scan at 23:53
Scanning 10.10.144.120 [1 port]
Completed ARP Ping Scan at 23:53, 0.04s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 23:53
Completed Parallel DNS resolution of 1 host. at 23:53, 0.01s elapsed
Initiating SYN Stealth Scan at 23:53
Scanning ip-10-10-144-120.eu-west-1.compute.internal (10.10.144.120) [1000 ports]
Discovered open port 22/tcp on 10.10.144.120
Discovered open port 80/tcp on 10.10.144.120
Completed SYN Stealth Scan at 23:53, 0.14s elapsed (1000 total ports)
Nmap scan report for ip-10-10-144-120.eu-west-1.compute.internal (10.10.144.120)
Host is up, received arp-response (0.0018s latency).
Scanned at 2021-10-13 23:53:02 UTC for 1s
Not shown: 998 closed ports
Reason: 998 resets
PORT   STATE SERVICE REASON
22/tcp open  ssh     syn-ack ttl 64
80/tcp open  http    syn-ack ttl 64
MAC Address: 02:E3:D7:60:03:A5 (Unknown)

Read data files from: /usr/bin/../shar

# All Ports:

In [10]:
!nmap -p- -sV -vv -oA  scans/tcp/tcp_scan $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-13 23:53 UTC
NSE: Loaded 45 scripts for scanning.
Initiating ARP Ping Scan at 23:53
Scanning 10.10.144.120 [1 port]
Completed ARP Ping Scan at 23:53, 0.05s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 23:53
Completed Parallel DNS resolution of 1 host. at 23:53, 0.00s elapsed
Initiating SYN Stealth Scan at 23:53
Scanning ip-10-10-144-120.eu-west-1.compute.internal (10.10.144.120) [65535 ports]
Discovered open port 22/tcp on 10.10.144.120
Discovered open port 80/tcp on 10.10.144.120
Completed SYN Stealth Scan at 23:53, 6.17s elapsed (65535 total ports)
Initiating Service scan at 23:53
Scanning 2 services on ip-10-10-144-120.eu-west-1.compute.internal (10.10.144.120)
Completed Service scan at 23:53, 11.06s elapsed (2 services on 1 host)
NSE: Script scanning 10.10.144.120.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 23:53
Completed NSE at 23:53, 0.02s elapsed
NSE: Starting runlevel 2 (of 2) scan.
Ini

In [11]:
import xmltodict

with open('scans/tcp/tcp_scan.xml','r') as f:
    xml_dict = xmltodict.parse(f.read())

In [12]:
PING=True
try:
    port_list = [port['@portid'] for port in xml_dict['nmaprun']['host']['ports']['port']]
    %env PORTS={",".join(port_list)}
except:
    print("Host is down")
    PING=False

env: PORTS=22,80


# No Ping:

In [13]:
!nmap -sV -Pn -vv -oA scans/no_ping/no_ping_scan $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-13 23:53 UTC
NSE: Loaded 45 scripts for scanning.
Initiating ARP Ping Scan at 23:53
Scanning 10.10.144.120 [1 port]
Completed ARP Ping Scan at 23:53, 0.04s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 23:53
Completed Parallel DNS resolution of 1 host. at 23:53, 0.00s elapsed
Initiating SYN Stealth Scan at 23:53
Scanning ip-10-10-144-120.eu-west-1.compute.internal (10.10.144.120) [1000 ports]
Discovered open port 22/tcp on 10.10.144.120
Discovered open port 80/tcp on 10.10.144.120
Completed SYN Stealth Scan at 23:53, 0.11s elapsed (1000 total ports)
Initiating Service scan at 23:53
Scanning 2 services on ip-10-10-144-120.eu-west-1.compute.internal (10.10.144.120)
Completed Service scan at 23:53, 11.05s elapsed (2 services on 1 host)
NSE: Script scanning 10.10.144.120.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 23:53
Completed NSE at 23:53, 0.01s elapsed
NSE: Starting runlevel 2 (of 2) scan.
Initi

In [14]:
if not PING:
    !nmap -sV -Pn -p- -vv -oA  scans/tcp/tcp_scan $TARGET

# Run Default Scripts:

In [15]:
if PING:
    !nmap -sC -p $PORTS -vv -oA scans/scripts/script_scan $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-13 23:53 UTC
NSE: Loaded 121 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 23:53
Completed NSE at 23:53, 0.00s elapsed
NSE: Starting runlevel 2 (of 2) scan.
Initiating NSE at 23:53
Completed NSE at 23:53, 0.00s elapsed
Initiating ARP Ping Scan at 23:53
Scanning 10.10.144.120 [1 port]
Completed ARP Ping Scan at 23:53, 0.03s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 23:53
Completed Parallel DNS resolution of 1 host. at 23:53, 0.00s elapsed
Initiating SYN Stealth Scan at 23:53
Scanning ip-10-10-144-120.eu-west-1.compute.internal (10.10.144.120) [2 ports]
Discovered open port 80/tcp on 10.10.144.120
Discovered open port 22/tcp on 10.10.144.120
Completed SYN Stealth Scan at 23:53, 0.04s elapsed (2 total ports)
NSE: Script scanning 10.10.144.120.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 23:53
Completed NSE at 23:53, 0.10s elapsed
NSE: Star

In [16]:
if PING:
    !nmap -A -T5 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T5 -Pn -vv --script=default,safe,discovery $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-13 23:53 UTC
NSE: Loaded 417 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 3) scan.
Initiating NSE at 23:53
NSE: [shodan-api] Error: Please specify your ShodanAPI key with the shodan-api.apikey argument
NSE: [broadcast-sonicwall-discover] No network interface was supplied, aborting.
NSE: [url-snarf] no network interface was supplied, aborting ...
NSE: [broadcast-ataoe-discover] No interface supplied, use -e
NSE: [targets-xml] Need to supply a file name with the targets-xml.iX argument
NSE: [targets-ipv6-map4to6] This script is IPv6 only.
NSE: [mtrace] A source IP must be provided through fromip argument.
NSE: [targets-ipv6-wordlist] Need to be executed for IPv6.
too short
too short
NSE Timing: About 99.00% done; ETC: 23:54 (0:00:00 remaining)
Completed NSE at 23:54, 40.01s elapsed
NSE: Starting runlevel 2 (of 3) scan.
Initiating NSE at 23:54
Completed NSE at 23:54, 0.00s elapsed
NSE: Starting runlevel 3 (

In [17]:
if PING:
    !nmap -A -T3 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T3 -Pn -vv --script=default,safe,discovery $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-14 00:03 UTC
NSE: Loaded 417 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 3) scan.
Initiating NSE at 00:03
NSE: [targets-xml] Need to supply a file name with the targets-xml.iX argument
NSE: [mtrace] A source IP must be provided through fromip argument.
NSE: [shodan-api] Error: Please specify your ShodanAPI key with the shodan-api.apikey argument
NSE: [broadcast-ataoe-discover] No interface supplied, use -e
NSE: [url-snarf] no network interface was supplied, aborting ...
NSE: [broadcast-sonicwall-discover] No network interface was supplied, aborting.
NSE: [targets-ipv6-wordlist] Need to be executed for IPv6.
NSE: [targets-ipv6-map4to6] This script is IPv6 only.
too short
too short
NSE Timing: About 97.00% done; ETC: 00:03 (0:00:01 remaining)
Completed NSE at 00:03, 40.05s elapsed
NSE: Starting runlevel 2 (of 3) scan.
Initiating NSE at 00:03
Completed NSE at 00:03, 0.00s elapsed
NSE: Starting runlevel 3 (

In [ ]:
from getpass import getpass
import os

if PING:
    !echo { getpass() } | sudo -S nmap -A -p- -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}
else:
    !echo { getpass() } | sudo -S nmap -A -p- -Pn -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}

In [ ]:
# UDP Scan, this takes forever...

# from getpass import getpass

# !echo { getpass() } > tmp.txt
# !cat tmp.txt | sudo -S nmap -sU --open -oA scans/udp/udp_scan $TARGET
# !cat tmp.txt | sudo -S nmap -sU --open -p- -oA scans/udp/udp_scan $TARGET
# !rm -rf tmp.txt